In [1]:
import os
import numpy as np
from skimage import io, color, transform, feature
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Path to the folders containing real and fake images
real_folder_path = "/content/gdrive/MyDrive/celebdf/original-cropped-images"
fake_folder_path = "/content/gdrive/MyDrive/celebdf/synthetic-cropped-images"

In [4]:
# Load and preprocess images
def load_and_preprocess_images(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder, filename)
            img = io.imread(img_path)
            img_gray = color.rgb2gray(img)
            img_resized = transform.resize(img_gray, (64, 64))  # Resize to a consistent size
            images.append(img_resized)
    return images

In [ ]:
real_images = load_and_preprocess_images(real_folder_path)
fake_images = load_and_preprocess_images(fake_folder_path)

In [ ]:
# Create labels (0 for real, 1 for fake)
real_labels = np.zeros(len(real_images))
fake_labels = np.ones(len(fake_images))

In [ ]:
# Combine real and fake data
all_images = np.concatenate((real_images, fake_images), axis=0)
all_labels = np.concatenate((real_labels, fake_labels), axis=0)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

In [5]:
# Extract HOG features
def extract_hog_features(images):
    hog_features = []
    for img in images:
        hog_feature = feature.hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        hog_features.append(hog_feature)
    return hog_features

In [ ]:
X_train_features = np.array(extract_hog_features(X_train))
X_test_features = np.array(extract_hog_features(X_test))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [6]:
# Define your neural network architecture
def build_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = Dense(512, activation='relu')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid')(x)  # Binary classification

    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
feature_dim = X_train_features.shape[1]
model = build_model((feature_dim,))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
batch_size = 32
epochs = 10

model.fit(X_train_features, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_features, y_test))

Epoch 1/10
750/750 [==============================] - 15s 9ms/step - loss: 0.5486 - accuracy: 0.7752 - val_loss: 0.3473 - val_accuracy: 0.8827
Epoch 2/10
750/750 [==============================] - 7s 10ms/step - loss: 0.3789 - accuracy: 0.8708 - val_loss: 0.3270 - val_accuracy: 0.8827
Epoch 3/10
750/750 [==============================] - 6s 8ms/step - loss: 0.3468 - accuracy: 0.8746 - val_loss: 0.3407 - val_accuracy: 0.8924
Epoch 4/10
750/750 [==============================] - 7s 10ms/step - loss: 0.3265 - accuracy: 0.8774 - val_loss: 0.3183 - val_accuracy: 0.8951
Epoch 5/10
750/750 [==============================] - 6s 8ms/step - loss: 0.3144 - accuracy: 0.8820 - val_loss: 0.3313 - val_accuracy: 0.8837
Epoch 6/10
750/750 [==============================] - 7s 10ms/step - loss: 0.3037 - accuracy: 0.8831 - val_loss: 0.2985 - val_accuracy: 0.8954
Epoch 7/10
750/750 [==============================] - 6s 8ms/step - loss: 0.2996 - accuracy: 0.8878 - val_loss: 0.2927 - val_accuracy: 0.8991
Ep

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_features, y_test)
print("Test accuracy:", test_accuracy)

188/188 [==============================] - 1s 3ms/step - loss: 0.2777 - accuracy: 0.9042
Test accuracy: 0.9042375683784485
